In [2]:
import pandas as pd
import numpy as np
import random
import plotly.express as px

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer

In [3]:
data = load_breast_cancer(as_frame=True)
X,y,df_bre = data.data,data.target,data.frame
diz_target = {0:'malignant',1:'benign'}
y = np.array([diz_target[y1] for y1 in y])
df_bre['target'] = df_bre['target'].apply(lambda x: diz_target[x])
df_bre.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,malignant
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,malignant
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,malignant
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,malignant
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,malignant


# 1: PCA implementation

In [6]:
class PCA_impl:
    def __init__(self,n_components,random_state=None,standardize=True):
        self.n_components = n_components
        self.random_state = random_state
        self.standardize = standardize

    def fit(self,X):

        self.X_copy = X.copy().astype('float32')

        if self.standardize==True:
            self.mean_ = np.mean(self.X_copy, axis=0)
            self.std_ = np.std(self.X_copy, axis=0)
            self.X_copy -= self.mean_
            self.X_copy /= self.std_

        if self.random_state ==True:
            random.seed(0)

        cov_mat = np.cov(self.X_copy.T)    
        eigen_values, eigen_vectors = np.linalg.eig(cov_mat)
        sorted_index = np.argsort(eigen_values)[::-1]
        self.sorted_eigenvalue_ = eigen_values[sorted_index]
        self.sorted_eigenvectors_ = eigen_vectors[:,sorted_index]
        self.explained_variance_ = self.sorted_eigenvalue_[0:self.n_components]
        total_var = self.sorted_eigenvalue_.sum()
        self.explained_variance_ratio_ = self.explained_variance_ / total_var
        self.cum_var_explained = np.cumsum(self.explained_variance_ratio_)

        self.projection_matrix_ = self.sorted_eigenvectors_[:,0:self.n_components]
        
        return

    def fit_transform(self,X,y):
        self.fit(X)
        self.components_ = np.dot(self.X_copy,self.projection_matrix_)
        self.df = pd.DataFrame(data=self.components_, columns=['PC{}'.format(i+1) for i in range(self.n_components)])
        self.df['label'] = y
        return self.components_

    def pca_plot2d(self):
        fig = px.scatter(self.components_, x=0, y=1, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show()

    def pca_plot3d(self):
        fig = px.scatter_3d(self.components_, x=0, y=1,z=2, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show()    

# 2: PCA without standardization

In [7]:
pca1 = PCA_impl(n_components=4,random_state=True,standardize=False)
X_new = pca1.fit_transform(X,y)
print('4D feature subspace:\n',X_new)
print('\nVariance Explained Ratio:\n',pca1.explained_variance_ratio_)
print('\nCumulative Variance Explained:\n',pca1.cum_var_explained)

4D feature subspace:
 [[2260.01388639 -187.96030082   17.91295273  -77.2629491 ]
 [2368.99375578  121.58742466  -66.05996862  -50.68968454]
 [2095.66520144  145.11398617  -32.37519053  -64.351628  ]
 ...
 [1414.37305601  153.5107474   -41.10784342  -78.32284775]
 [2224.72942843  140.08646838  -50.40752361  -92.21185471]
 [ 328.34369582   17.31413276   -6.77640537  -66.00371815]]

Variance Explained Ratio:
 [9.82044672e-01 1.61764899e-02 1.55751071e-03 1.20931967e-04]

Cumulative Variance Explained:
 [0.98204467 0.99822116 0.99977867 0.9998996 ]


In [8]:
pca1.pca_plot2d()

In [9]:
pca1.pca_plot3d()

# 3: PCA with standardization

In [10]:
pca1 = PCA_impl(n_components=4,random_state=True,standardize=True)
X_new = pca1.fit_transform(X,y)
print('4D feature subspace:\n',X_new)
print('\nVariance Explained Ratio:\n',pca1.explained_variance_ratio_)
print('\nCumulative Variance Explained:\n',pca1.cum_var_explained)

4D feature subspace:
 [[ 9.19283657  1.94858245 -1.12316625  3.63373111]
 [ 2.38780138 -3.76817203 -0.52929263  1.118264  ]
 [ 5.73389601 -1.07517422 -0.55174749  0.91208288]
 ...
 [ 1.25617898 -1.90229693  0.5627307  -2.08922671]
 [10.37479391  1.67200965 -1.87702939 -2.35603056]
 [-5.47524347 -0.67063702  1.49044308 -2.29915719]]

Variance Explained Ratio:
 [0.44272025 0.18971182 0.09393163 0.06602135]

Cumulative Variance Explained:
 [0.44272025 0.63243207 0.72636371 0.79238506]


In [11]:
pca1.pca_plot2d()

In [12]:
pca1.pca_plot3d()

# PCA with sklearn

In [17]:
pca = PCA(n_components=4,random_state=123)
components = pca.fit_transform(X)
print('4D feature subspace:\n',components)
df = pd.DataFrame(data=components, columns=['PC{}'.format(i) for i in range(components.shape[1])])
# plot
df['label'] = y
print('\nVariance Explained Ratio:\n',pca.explained_variance_ratio_)
print('\nCumulative Variance Explained:\n',np.cumsum(pca.explained_variance_ratio_))

4D feature subspace:
 [[1160.1425737  -293.91754364   48.57839763   -8.71197531]
 [1269.12244319   15.63018184  -35.39453423   17.86128323]
 [ 995.79388896   39.15674324   -1.70975298    4.1993401 ]
 ...
 [ 314.50175618   47.55352518  -10.44240718   -9.77188104]
 [1124.85811531   34.12922497  -19.74208742  -23.66088144]
 [-771.52762188  -88.64310636   23.88903189    2.54724855]]

Variance Explained Ratio:
 [9.82044672e-01 1.61764899e-02 1.55751075e-03 1.20931964e-04]

Cumulative Variance Explained:
 [0.98204467 0.99822116 0.99977867 0.9998996 ]


In [18]:
fig = px.scatter(components, x=0, y=1, color=df.label,labels={'0': 'PC 1', '1': 'PC 2'})
fig.show() 

In [19]:
fig = px.scatter_3d(components, x=0, y=1,z=2, color=df.label,labels={'0': 'PC 1', '1': 'PC 2','2':'PC 3'})
fig.show() 

In [36]:
X_copy = X.copy().astype('float32')
#X_copy -= np.mean(X_copy, axis=0)
X_copy /= np.std(X_copy, axis=0)

In [37]:
pca = PCA(n_components=4,random_state=123)
components = pca.fit_transform(X_copy)
print('4D feature subspace:\n',components)
df = pd.DataFrame(data=components, columns=['PC{}'.format(i) for i in range(components.shape[1])])
# plot
df['label'] = y
print('\nVariance Explained Ratio:\n',pca.explained_variance_ratio_)
print('\nCumulative Variance Explained:\n',np.cumsum(pca.explained_variance_ratio_))

4D feature subspace:
 [[ 9.192833   1.948584  -1.1231629  3.6337385]
 [ 2.3878028 -3.7681692 -0.5292903  1.1182367]
 [ 5.7338977 -1.075172  -0.5517465  0.9120648]
 ...
 [ 1.2561802 -1.9022965  0.5627277 -2.0892234]
 [10.374793   1.6720114 -1.8770341 -2.3560226]
 [-5.475243  -0.6706354  1.4904404 -2.2991896]]

Variance Explained Ratio:
 [0.44272023 0.1897118  0.09393165 0.06602137]

Cumulative Variance Explained:
 [0.44272023 0.63243204 0.7263637  0.7923851 ]


In [38]:
fig = px.scatter(components, x=0, y=1, color=df.label,labels={'0': 'PC 1', '1': 'PC 2'})
fig.show() 

In [35]:
fig = px.scatter_3d(components, x=0, y=1,z=2, color=df.label,labels={'0': 'PC 1', '1': 'PC 2','2':'PC 3'})
fig.show() 